In [1]:
## Loading relevant packages
library("slackr");library(AirSensor);library(MazamaSpatialUtils)
library(lubridate);library(dplyr);library(ggmap)
library(texPreview) ## This package is not mandatory so ignore if you can't install it


Loading required package: sp

Warning message in fun(libname, pkgname):
“rgeos: versions of GEOS runtime 3.9.1-CAPI-1.14.2
and GEOS at installation 3.9.1dev-CAPI-1.14.1differ”
Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggplot2

Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.

Please cite ggmap if you use it! See citation("ggmap") for details.



In [3]:
## Loading Spatial data 
## More details on Air Sensor Git hub page if it's not setup already
initializeMazamaSpatialUtils("Spatial_data")

In [4]:
## Settng the Country Code API link and look back days 
pas <- pas_createNew(countryCodes = "IN",baseUrl = 'https://www.purpleair.com/json?all=true',
                     includePWFSL = F)
## Changing the time zone to Asia/Kolkata
pas$lastSeenDate <- with_tz(pas$lastSeenDate,tz="Asia/Kolkata")
## Selecting ONLY NASA Sensors
pas <- pas %>% pas_filter(grepl('NASA',label),!grepl("B",label))


In [2]:
## Setting the slack system more details on slackr git hub page
slackr_setup(channel = "pa_sensor_alert",
                 bot_user_oauth_token = "xxxx",
                 incoming_webhook_url = "xxx")


[1] "Successfully connected to Slack"

In [6]:
#Selecting specific columns from the pas dataframe
columns <- c("label",'latitude',"longitude",'lastSeenDate','humidity','temperature','pm25_current','pm25_1hr',
             'pm25_6hr','pm25_1day','pm25_1week')
pas_filter <- pas[,columns]
str(pas_filter)

pa_synoptic[,11] [31 × 11] (S3: pa_synoptic/pa_synoptic/tbl_df/tbl/data.frame)
 $ label       : chr [1:31] "NASA-AQCS-134" "NASA_AQCS-178" "NASA_AQCS_101" "NASA_AQCS_102" ...
 $ latitude    : num [1:31] 24.8 28.6 31 30.9 28.5 ...
 $ longitude   : num [1:31] 85 77.1 75.7 75.8 77.5 ...
 $ lastSeenDate: POSIXct[1:31], format: "2021-04-04 22:22:01" "2021-04-05 11:58:35" ...
 $ humidity    : num [1:31] 8 10 23 16 11 NA 8 16 10 16 ...
 $ temperature : num [1:31] 103 101 92 102 104 NA 107 100 105 102 ...
 $ pm25_current: num [1:31] 46.1 26.6 45.5 46.5 40 ...
 $ pm25_1hr    : num [1:31] 48.2 43.4 60.8 68.5 97.6 ...
 $ pm25_6hr    : num [1:31] 81.7 46.9 59.4 66.7 91.3 ...
 $ pm25_1day   : num [1:31] 83.9 41.6 48.5 53.8 73.9 ...
 $ pm25_1week  : num [1:31] 123.2 65.3 60.8 68 105.6 ...


In [7]:
## Looking at first ten rows from the dataset 
## This is where we can put filter on the rows or sensors based on our need
subset_pas_filter <- pas_filter[1:10,]
## Time at which the data is being query
time <- Sys.time()
## Changing the time zone to UTC
time <- with_tz(time,tz="Asia/Kolkata")
slackr("Time of Query",time)
## You can use knitr or Pandoc to send neat tables. Install the package from the cron
#slackr("Sensor basic Info", knitr::kable(subset_pas_filter, format = "markdown"))
slackr("Sensor basic Info", pander::pandoc.table(subset_pas_filter))

In [8]:
## This is used to get location  (Add your google map API key)
ggmap::register_google(key = "xxxx")
#location <- do.call(rbind,
#                  lapply(1:nrow(subset_pas_filter),
#                         function(i)revgeocode(as.numeric(subset_pas_filter[i,3:2]))))


In [9]:
## Another approach is to save the files as .csv and then upload it in .csv to slack 
file_name = "NASA_Sensor_Status_2021_03_03_30.csv" ## This has to be generated automatically so use paste0 to generate
## Saving the .csv files
write.csv(pas_filter,paste0("Files/",file_name),row.names=F)
## Now will upload the .csv files
slackr_upload(filename = paste0("Files/",file_name),title = file_name)

In [3]:
readRenviron("~/Git/API.Renviron")

In [7]:
source("~/Git/Sensor_Alert/Message_Alert.R")

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”


In [8]:
getwd()

[1] "/home/jovyan/Git/Sensor_Alert"